# macrotools

In [2]:
"""
    postwalk(f, expr)
Applies `f` to each node in the given expression tree, returning the result.
`f` sees expressions *after* they have been transformed by the walk. See also
`prewalk`.
"""
postwalk(f, x) = walk(x, x -> postwalk(f, x), f)

"""
    prewalk(f, expr)
Applies `f` to each node in the given expression tree, returning the result.
`f` sees expressions *before* they have been transformed by the walk, and the
walk will be applied to whatever `f` returns.
This makes `prewalk` somewhat prone to infinite loops; you probably want to try
`postwalk` first.
"""
prewalk(f, x)  = walk(f(x), x -> prewalk(f, x), identity)

replace(ex, s, s′) = prewalk(x -> x == s ? s′ : x, ex)

"""
    inexpr(expr, x)
Simple expression match; will return `true` if the expression `x` can be found
inside `expr`.
    inexpr(:(2+2), 2) == true
"""
function inexpr(ex, x)
  result = false
  postwalk(ex) do y
    if y == x
      result = true
    end
    return y
  end
  return result
end

inexpr

# modeltool

In [3]:
"""    bodyblock(expr::Expr)
get the array of args representing the body of a function definition.
"""
function bodyblock(expr::Expr)
    expr.head == :function || error("$expr is not a function definition")
    return expr.args[2].args
end

"""    funclines(expr::Expr, s::Symbol)
clean up the lines of a function definition for presentation
"""
function funclines(expr::Expr, s::Symbol)
    q = Expr(:block)
    isexpr(x) = isa(x, Expr)
    q.args = (filter(isexpr, findfunc(expr, s))[end]
              |> bodyblock
              |> arr -> filter(x->!isa(x, LineNumberNode),arr))
    return q
end
"""    argslist(expr::Expr)
get the array of args representing the arguments of a defined function.
the first element of this list is the function name
See also [`bodyblock`](@ref), [`pusharg!`](@ref),
"""
function argslist(expr::Expr)
    expr.head == :function || error("$expr is not a function definition")
    return expr.args[1].args
end

"""    pusharg!(expr::Expr, s::Symbol)
push a new argument onto the definition of a function.
See also [`argslist`](@ref), [`setarg!`](@ref)
"""
function pusharg!(ex::Expr, s::Symbol)
    ex.head == :function || error("ex is not a function definition")
    push!(argslist(ex), s)
    return ex
end

"""    setarg!(expr::Expr, s::Symbol)
replace the argument in a function call.
See also [`argslist`](@ref), [`pusharg!`](@ref)
"""
function setarg!(ex::Expr, old, new)
    ex.head == :call || error("ex is not a function call")
    for (i, x) in enumerate(ex.args)
        if x == old
            ex.args[i] = new
        end
    end
    return ex
end

setarg!

# findfunc

In [4]:
"""    findfunc(expr::Expr, name::Symbol)

findfunc walks the AST of `expr` to find the definition of function called `name`.

This function returns a reference to the original expression so that you can modify it inplace
and is intended to help users rewrite the definitions of functions for generating new models.
"""
function findfunc(expr::Expr, name::Symbol)
    try
        expr.head
    catch
        return nothing
    end

    if expr.head == :module
        return findfunc(expr.args[3], name)
    end
    if expr.head == :function
        # normal function definition syntax ie. function f(x); return y end
        if expr.args[1].args[1] == name
            return expr
        end
        return findfunc(expr.args, name)
    end
    if expr.head == :(=)
        if isa(expr.args[1], Symbol)
            return nothing
        end

        if expr.args[1].head == :call
            # inline function definition ie. f(x) = y
            if expr.args[1].args[1] == name
                return expr
            end
            return findfunc(expr.args, name)
        end
    end
    if expr.head == :block
        return findfunc(expr.args, name)
    end
    return nothing
end

function findfunc(expr::LineNumberNode, s::Symbol)
    return nothing
end

function findfunc(args::Vector{Any}, name::Symbol)
    return filter(x->x!=nothing, [findfunc(a,name) for a in args])
end


walk(x, inner, outer) = outer(x)
walk(x::Expr, inner, outer) = outer(Expr(x.head, map(inner, x.args)...))

"""    findassign(expr::Expr, name::Symbol)

findassign walks the AST of `expr` to find the assignments to a variable called `name`.

This function returns a reference to the original expression so that you can modify it inplace
and is intended to help users rewrite expressions for generating new models.

See also: [`findfunc`](@ref).
"""
function findassign(expr::Expr, name::Symbol)
    # g(y) = filter(x->x!=nothing, y)
    matches = Expr[]
    g(y::Any) = :()
    f(x::Any) = :()
    f(x::Expr) = begin
        if x.head == :(=)
            if x.args[1] == name
                push!(matches, x)
                return x
            end

        end
        walk(x, f, g)
    end
    walk(expr, f, g)
    return matches
end

function replacevar(expr::Expr, name::Symbol, newname::Symbol)
    g(x::Any) = x
    f(x::Any) = x
    f(x::Symbol) = (x==name ? newname : x)
    f(x::Expr) = walk(x, f, g)
    return walk(expr, f, g)
end

function replacevar(expr::Expr, tr::Dict{Symbol, Any})
    g(x::Any) = x
    f(x::Any) = x
    f(x::Symbol) = get(tr, x, x)
    f(x::Expr) = walk(x, f, g)
    return walk(expr, f, g)
end

replacevar (generic function with 2 methods)

# parsers

In [5]:
using Base.Meta
import Base.push!

"""    parsefile(path)

read in a julia source file and parse it.

Note: If the top level is not a simple expression or module definition the file is wrapped in a Module named modprefix.
"""
function parsefile(path, modprefix="Modeling")
    s = read(path, String)
    # open(path) do fp
    #     s = read(String, fp)
    # end
    try
        expr = Base.Meta.parse(s)
        return expr
    catch
        s = "module $modprefix\n$s \nend #module $modprefix"
        expr = Base.Meta.parse(s)
        return expr
    end
end

"""    AbstractCollector

subtypes of AbstractCollector support extracting and collecting information
from input sources.
"""
abstract type AbstractCollector end


"""    FuncCollector{T} <: AbstractCollector

collects function definitions and names
"""
struct FuncCollector{T} <: AbstractCollector
    defs::T
end

function push!(fc::AbstractCollector, expr::LineNumberNode)
    return nothing
end

function push!(fc::FuncCollector, expr::Expr)
    if expr.head == :function
        push!(fc.defs, expr.args[1] => expr.args[2])
    end
end

"""   MetaCollector{T,U,V,W} <: AbstractCollector

collects multiple pieces of information such as

- exprs: expressions
- fc: functions
- vc: variable assignments
- modc: module imports
"""
struct MetaCollector{T,U,V,W} <: AbstractCollector
    exprs::V
    fc::T
    vc::U
    modc::W
end

function push!(mc::MetaCollector, expr::Expr)
    push!(mc.exprs, expr)
    push!(mc.fc, expr)
    if expr.head == :(=)
        @debug "pushing into vc" expr=expr
        push!(mc.vc, expr.args[1]=>expr.args[2])
    elseif expr.head == :using
        push!(mc.modc, expr.args[1].args)
    else
        @info "unknown expr type for metacollector"
        @show expr
    end
end


"""    funcs(body)

collect the function definitions from a module expression.
"""
function funcs(body)
    fs = FuncCollector([])
    for subexpr in body
        push!(fs, subexpr)
    end
    return fs
end

"""    defs(body)

collect the function definitions and variable assignments from a module expression.
"""
function defs(body)
    # fs = funcs(body)
    mc = MetaCollector(Any[], FuncCollector([]), Any[], Any[])
    for expr in body
        push!(mc, expr)
    end
    return mc
end

function recurse(mc::AbstractCollector)
    subdefs = Any[]
    funcdefs = mc.fc.defs
    @show funcdefs
    for def in funcdefs
        funcname = def[1]
        funcquote = def[2]
        push!(subdefs, funcname=>defs(funcquote.args))
    end
    return subdefs
end

recurse (generic function with 1 method)

# call finder 

In [35]:
s = 
"
module test
   function a(x)
       return x+2
   end
   function b(y)
      return y/2
   end
   function main(x)
       println(b(a(x)))
   end
   main(1)
end
"
ex = Meta.parse(s)
dump(ex)

Expr
  head: Symbol module
  args: Array{Any}((3,))
    1: Bool true
    2: Symbol test
    3: Expr
      head: Symbol block
      args: Array{Any}((9,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol none
        2: LineNumberNode
          line: Int64 3
          file: Symbol none
        3: Expr
          head: Symbol function
          args: Array{Any}((2,))
            1: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol a
                2: Symbol x
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr
        4: LineNumberNode
          line: Int64 6
          file: Symbol none
        5: Expr
          head: Symbol function
          args: Array{Any}((2,))
            1: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol b
                2: Symbol y
            

# goal code

In [152]:
s = "
module test
d = Dict()
function add(a,b)
    push!(d,(:a=>typeof(a)))
    push!(d,(:b=>typeof(b)))
    a + b
end
add(1,2)
push!(d,(:add=>typeof(add(1,2))))
end
"
ex = Meta.parse(s)
dump(ex)

Expr
  head: Symbol module
  args: Array{Any}((3,))
    1: Bool true
    2: Symbol test
    3: Expr
      head: Symbol block
      args: Array{Any}((9,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol none
        2: LineNumberNode
          line: Int64 3
          file: Symbol none
        3: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol d
            2: Expr
              head: Symbol call
              args: Array{Any}((1,))
                1: Symbol Dict
        4: LineNumberNode
          line: Int64 4
          file: Symbol none
        5: Expr
          head: Symbol function
          args: Array{Any}((2,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol add
                2: Symbol a
                3: Symbol b
            2: Expr
              head: Symbol block
              args: Array{Any}((6,))
                1: LineNumberNode
              

In [1]:
s = "
module test
function add(a,b)
    a + b
end
println(add(1,2))
end
"
e = Meta.parse(s)
@show e
dump(e)

e = :(module test
  #= none:2 =#
  #= none:3 =#
  function add(a, b)
      #= none:4 =#
      a + b
  end
  #= none:6 =#
  println(add(1, 2))
  end)
Expr
  head: Symbol module
  args: Array{Any}((3,))
    1: Bool true
    2: Symbol test
    3: Expr
      head: Symbol block
      args: Array{Any}((5,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol none
        2: LineNumberNode
          line: Int64 3
          file: Symbol none
        3: Expr
          head: Symbol function
          args: Array{Any}((2,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol add
                2: Symbol a
                3: Symbol b
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr
        4: LineNumberNode
          line: Int64 6
          file: Symbol none
        5: Expr
          head: Symbol call
          args:

In [2]:
function recurser(expr::Expr)
    out = Expr(expr.head)
    for internal in expr.args
        if typeof(internal) == Expr
            if internal.head == :function
                # push!(internal.args,:(push!(edges) ))
                push!(out.args,internal)
            else
                push!(out.args,recurser(internal))
            end
        else
            push!(out.args,internal)
        end
    end #for
    return out
end # recurser 

recurser (generic function with 1 method)

In [3]:
function typegraph(expr::Expr)
    out = Expr(expr.head)
    push!(out.args,expr.args[1])
    push!(out.args,expr.args[2])
    push!(out.args,Expr(:block))
    push!(out.args[3].args,:(edges=Dict()))
    push!(out.args[3].args,recurser(expr.args[3]))
    return out
end

d = typegraph(e)
# dump(d)

:(module test
  edges = Dict()
  begin
      #= none:2 =#
      #= none:3 =#
      function add(a, b)
          #= none:4 =#
          a + b
      end
      #= none:6 =#
      println(add(1, 2))
  end
  end)

In [23]:
function check(expr::Expr)
    for arg in expr.args
        if typeof(arg) == Expr
            for inside in arg.args
                try
                    if inside.head == :call
                        return inside.args[1](inside.args[2:end]...)
                        end # if
                    return inside.args[1](inside.args[2:end]...)
                catch
                    continue
                end  # try
            end
            return inside.args[1](inside.args[2:end]...)
        end
    end
    return inside.args[1](inside.args[2:end]...)
end

check (generic function with 1 method)

In [24]:
check(e)

UndefVarError: UndefVarError: inside not defined

In [21]:
f = (2,3,2,4)

(2, 3, 2, 4)